## MLflow's Model Registry

In [6]:
from mlflow.tracking import MlflowClient


#MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
MLFLOW_TRACKING_URI="http://127.0.0.1:5000"



### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [7]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/artifacts/1', creation_time=1765782817531, experiment_id='1', last_update_time=1765782817531, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/mlruns/0', creation_time=1765781523578, experiment_id='0', last_update_time=1765781523578, lifecycle_stage='active', name='Default', tags={'mlflow.experimentKind': 'custom_model_development'}>]

In [3]:
client.get_experiment_by_name("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/artifacts/1', creation_time=1765782817531, experiment_id='1', last_update_time=1765782817531, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>

Let's check the latest versions for the experiment with id `1`...

In [10]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    #filter_string="metrics.rmse > 7",
    #run_view_type=ViewType.ACTIVE_ONLY,
   # max_results=5,
    order_by=["metrics.rmse ASC"]
)
print(runs)

[<Run: data=<RunData: metrics={'rmse': 39.92275724413357}, params={'learning_rate': '0.09585355369315604',
 'max_depth': '30',
 'min_child_weight': '1.060597050922164',
 'objective': 'reg:linear',
 'reg_alpha': '0.018060244040060163',
 'reg_lambda': '0.011658731377413597',
 'seed': '42'}, tags={'mlflow.runName': 'efficient-mule-118',
 'mlflow.source.name': '/workspaces/mlops-zoomcamp/.venv/lib/python3.12/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'codespace'}>, info=<RunInfo: artifact_uri='/workspaces/mlops-zoomcamp/artifacts/1/9d15362c70444ed78c8db9fd468d6e73/artifacts', end_time=1765853235344, experiment_id='1', lifecycle_stage='active', run_id='9d15362c70444ed78c8db9fd468d6e73', run_name='efficient-mule-118', start_time=1765853119783, status='FINISHED', user_id='codespace'>, inputs=<RunInputs: dataset_inputs=[], model_inputs=[]>, outputs=<RunOutputs: model_outputs=[<LoggedModelOutput: model_id='m-2b8ee5fcb9564e1989e47c6a1abeb920', step=0>]>

In [11]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 9d15362c70444ed78c8db9fd468d6e73, rmse: 39.9228
run id: d27ce45bad8a423e944b24748bcba895, rmse: 40.2264
run id: 9047b474b0b141558bf125a1390ed0bd, rmse: 40.8487
run id: 522d965de4584d1daceab27d6bdde8f8, rmse: 41.0182
run id: a84b381e67f344f68781ee76b2e15535, rmse: 41.2893
run id: 661e149caad543fc90d9792219b45202, rmse: 41.6723
run id: 99d6520b5a0e48528d4c30721c9424e8, rmse: 41.7485
run id: f1611ab7eaac4f93b5fdd06a8bf5c4d5, rmse: 41.9883
run id: fca8757cc564495ba85f23a196ef032c, rmse: 42.2211
run id: 9fdf40148db147e3a015ebaee8b962c3, rmse: 42.7286
run id: 772dacac93374c068b96c09d3a40d4a2, rmse: 43.0279
run id: cc670d479c16424db101bbaaadaf5cdd, rmse: 43.8938
run id: 362881294b564257b3de6fec07f8ada6, rmse: 43.8987
run id: 6da0e2dc163448dd96e7abe2eb544b3c, rmse: 45.4587
run id: 1c72cfc8b206433fab44a915a652a853, rmse: 47.7839
run id: b4a4e580d9d5417ebdd90fb5e1d0d662, rmse: 48.0694
run id: d571e671547f4fbcbfbc938987635c48, rmse: 147.4581
run id: 2a4d5a135f08450e8cb4c37f00f53294, rmse:

KeyError: 'rmse'

### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [13]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [14]:
run_id = "f1611ab7eaac4f93b5fdd06a8bf5c4d5"
model_uri = f"runs:/{run_id}/models_mlflow"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2025/12/17 06:23:11 WARNING mlflow.tracking._model_registry.fluent: Run with id f1611ab7eaac4f93b5fdd06a8bf5c4d5 has no artifacts at artifact path 'models_mlflow', registering model based on models:/m-3fa19771ae6c4fc587b1450c3815a44c instead
2025/12/17 06:23:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 2
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1765952591345, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1765952591345, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='f1611ab7eaac4f93b5fdd06a8bf5c4d5', run_link='', source='models:/m-3fa19771ae6c4fc587b1450c3815a44c', status='READY', status_message=None, tags={}, user_id='', version='2'>

In [20]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: None


/tmp/ipykernel_18844/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [12]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_10220/1600074043.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1765951813253, current_stage='Staging', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1765951923150, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='f1611ab7eaac4f93b5fdd06a8bf5c4d5', run_link='', source='models:/m-3fa19771ae6c4fc587b1450c3815a44c', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [13]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1765951813253, current_stage='Staging', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='The model version 1 was transitioned to Staging on 2025-12-17', last_updated_timestamp=1765951940083, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='f1611ab7eaac4f93b5fdd06a8bf5c4d5', run_link='', source='models:/m-3fa19771ae6c4fc587b1450c3815a44c', status='READY', status_message=None, tags={}, user_id='', version='1'>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [2]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred)}  #, squared=False)}

In [3]:
df = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet")

In [15]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [16]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [17]:
X_test = preprocess(df, dv)

In [18]:
target = "duration"
y_test = df[target].values

In [22]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 3.28 s, sys: 97 ms, total: 3.38 s
Wall time: 5.42 s


{'rmse': 41.143856174527926}

In [25]:
%time test_model(name=model_name, stage=None, X_test=X_test, y_test=y_test)

CPU times: user 2.58 s, sys: 6.68 ms, total: 2.59 s
Wall time: 838 ms


{'rmse': 41.143856174527926}

In [26]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_18844/1316468422.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1765951813253, current_stage='Production', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='The model version 1 was transitioned to Staging on 2025-12-17', last_updated_timestamp=1765952799147, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='f1611ab7eaac4f93b5fdd06a8bf5c4d5', run_link='', source='models:/m-3fa19771ae6c4fc587b1450c3815a44c', status='READY', status_message=None, tags={}, user_id='', version='1'>